# 1: Import Packages and Define Functions

In [594]:
# Import some libraries that will be used
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
import sys
sys.path.insert(1, '/Users/brianmccabe/DataScience/Flatiron/mod5/Emoji_Analysis/Scripts/')
import config



pd.set_option('display.max_columns', 300)

%matplotlib inline

In [595]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import cm
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
import string
import scipy
import emoji
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brianmccabe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/brianmccabe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/brianmccabe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [596]:
def extract_emojis(s):
    return ''.join(c for c in s if c in emoji.UNICODE_EMOJI)

In [597]:
def check_for_flags(s):
    s = str(s)
    has_flag = False
    for flag in flags:
        if flag in s:
            has_flag = True
    return has_flag

In [598]:
def unique_emoji_count(s):
    return len(set([c for c in s]))

In [599]:
def condence_emojis(s):
    try:
        return set(s).pop()
    except:
        return None

In [600]:
analyzer = SentimentIntensityAnalyzer()
def sentiment_scores(s):
    return analyzer.polarity_scores(s)['compound']

In [601]:
def remove_emojis(s):
    pattern = "\w+"
    return ' '.join(re.findall(pattern, s))

In [625]:
def impute_top_emoji(s, mappings = emoji_sentiment_mappings, data = df):
    val = df[df.emoji == s].sentiment_score.values[0]
    s = list(set(s))
    closest = 100
    top = s[0]
    for i in range(0, len(s)):
        try:
            if abs(val - emoji_sentiment_mappings.loc[s[i]].sentiment_score) < closest:
                closest = abs(val - emoji_sentiment_mappings.loc[s[i]].sentiment_score)
                top = s[i]
        except:
            continue
    return top

# 2: Load in the Data

In [602]:
connection = mysql.connector.connect(host=config.host,
                                         user=config.user,
                                         port=config.port,
                                         password=config.password,
                                         database=config.database,
                                         auth_plugin='mysql_native_password')

cursor = connection.cursor()
query = "SELECT tweet FROM Tweets WHERE HEX(tweet) RLIKE '^(..)*F.'"

df = pd.read_sql(query, connection)

In [603]:
df['emoji'] = df.tweet.apply(extract_emojis)

In [604]:
df.head()

,tweet,emoji
0,""" cause hm? well, maybe \nchange is needed. yo...",
1,""" well, i will 𝙨𝙩𝙞𝙡𝙡 thank you for it. ~\n\na...",
2,"""'One day, honest citizens are going to stand ...",🤣
3,"""...You are not alone.\nThere's a place where ...",🎶🎶🎼
4,"""5 Books That Changed My Life As An Investor""....",👏


In [605]:
# remove white spaces in emoji column since they mess up unique emoji count below...
df.emoji.apply(str.strip).loc[9]

'🤣😭'

In [606]:
# We can see that emoji flags have strange representations. I will remove them from the dataset.
[c for c in df.emoji.loc[9]]

['🤣', '😭']

In [607]:
import flag
from bs4 import BeautifulSoup
import requests

In [608]:
url = 'https://en.wikipedia.org/wiki/Regional_indicator_symbol'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [609]:
flags = []
skip_first = 0
for item in soup.find('tbody').find_all('tr'):
    if skip_first == 0:
        skip_first += 1
        continue
    flagcode = item.find_all('td')[1].text
    flags.append(flag.flag(flagcode))

flags[0:5]    

['🇦🇨', '🇦🇩', '🇦🇪', '🇦🇫', '🇦🇬']

In [610]:
df.emoji.apply(check_for_flags).value_counts()

False    27093
True       989
Name: emoji, dtype: int64

In [611]:
df['has_flag'] = df.emoji.apply(check_for_flags)
df = df[df.has_flag == False]
df.drop('has_flag', axis=1, inplace=True)

In [612]:
df['unique_emoji_count'] = df.emoji.apply(unique_emoji_count)

In [613]:
df.head()

,tweet,emoji,unique_emoji_count
0,""" cause hm? well, maybe \nchange is needed. yo...",,0
1,""" well, i will 𝙨𝙩𝙞𝙡𝙡 thank you for it. ~\n\na...",,0
2,"""'One day, honest citizens are going to stand ...",🤣,1
3,"""...You are not alone.\nThere's a place where ...",🎶🎶🎼,2
4,"""5 Books That Changed My Life As An Investor""....",👏,1


In [614]:
df['sentiment_score'] = df.tweet.apply(sentiment_scores)

In [615]:
df.head()

,tweet,emoji,unique_emoji_count,sentiment_score
0,""" cause hm? well, maybe \nchange is needed. yo...",,0,0.3818
1,""" well, i will 𝙨𝙩𝙞𝙡𝙡 thank you for it. ~\n\na...",,0,0.8555
2,"""'One day, honest citizens are going to stand ...",🤣,1,0.8288
3,"""...You are not alone.\nThere's a place where ...",🎶🎶🎼,2,-0.0926
4,"""5 Books That Changed My Life As An Investor""....",👏,1,0.5719


In [616]:
df['tweet'] = df.tweet.apply(remove_emojis)

In [617]:
df.head()

,tweet,emoji,unique_emoji_count,sentiment_score
0,cause hm well maybe change is needed you ve sh...,,0,0.3818
1,well i will 𝙨𝙩𝙞𝙡𝙡 thank you for it and i can g...,,0,0.8555
2,One day honest citizens are going to stand up ...,🤣,1,0.8288
3,You are not alone There s a place where FEAR h...,🎶🎶🎼,2,-0.0926
4,5 Books That Changed My Life As An Investor Ex...,👏,1,0.5719


In [618]:
#df['emoji'] = df.emoji.apply(condence_emojis)

In [619]:
df.emoji.isna().sum()

0

In [620]:
df.dropna(inplace=True)

In [621]:
df.isna().sum()

tweet                 0
emoji                 0
unique_emoji_count    0
sentiment_score       0
dtype: int64

In [622]:
df = df[df.unique_emoji_count > 0]

In [623]:
df.reset_index(inplace=True)

In [626]:
# Creating a dataframe of singular emojis to the average sentiment score they obtain
df2 = df.copy()
df2 = df2[df2.unique_emoji_count == 1]
df2['emoji'] = df2.emoji.apply(condence_emojis)
emoji_sentiment_mappings = df2[['emoji', 'sentiment_score']].groupby('emoji').mean()

In [632]:
df['top_emoji'] = df.emoji.apply(impute_top_emoji)

In [634]:
df.drop(["unique_emoji_count", "emoji"], axis=1, inplace=True)
df.drop('index', axis=1, inplace=True)

In [642]:
emoji_sentiment_mappings.sort_values(by='sentiment_score')[0:10]

,sentiment_score
emoji,
🕵,-0.9300
🐇,-0.8830
🚁,-0.8760
🍵,-0.8618
👮,-0.8225
🇫,-0.8074
🏳,-0.8034
👐,-0.7885
🥚,-0.7509


In [650]:
emoji_counts = dict(df.top_emoji.value_counts())

In [654]:
df['emoji_frequency'] = df.top_emoji.map(emoji_counts)

In [657]:
df

,tweet,sentiment_score,top_emoji,emoji_frequency
0,One day honest citizens are going to stand up ...,0.8288,🤣,988
1,You are not alone There s a place where FEAR h...,-0.0926,🎶,63
2,5 Books That Changed My Life As An Investor Ex...,0.5719,👏,169
3,8616 new cases in 24 hours it s scary stay saf...,-0.0772,🤍,56
4,absolutely losing it over this picture of my b...,0.7187,💕,206
...,...,...,...,...
26962,IncredibleEdibleWrecsam would Wrexham folk to ...,0.9899,💚,84
26963,IncredibleEdibleWrecsam would Wrexham folk to ...,0.9899,💚,84
26964,LOL no sympathy for ChickfilA enjoy your loote...,0.9943,🤣,988
26965,Says the person who is able to walk past the d...,0.9911,😂,2717
